# Step 1 – Environment & Basic Configuration

This notebook:

- Imports core libraries
- Checks PyTorch version
- Detects whether GPU (CUDA) is available


In [1]:
# Step 1: Imports & Basic Configuration

import os
from typing import List, Tuple, Dict

import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

print("PyTorch version:", torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


PyTorch version: 2.9.1+cpu
Using device: cpu


# Step 2 – Data Paths & Dummy ArchiveII-Style CSV

This notebook:

- Defines base data directories
- Sets up an `archiveII/processed/` folder
- Creates a small **dummy CSV** that mimics ArchiveII format:
  - `sequence_id`
  - `sequence`
  - `dot_bracket`

You will later replace the dummy CSV with real processed ArchiveII data.


In [2]:
import os
import pandas as pd

BASE_DATA_DIR = "./data"
ARCHIVEII_DIR = os.path.join(BASE_DATA_DIR, "archiveII")
PROCESSED_DIR = os.path.join(ARCHIVEII_DIR, "processed")

os.makedirs(PROCESSED_DIR, exist_ok=True)
print("Processed dir:", PROCESSED_DIR)


Processed dir: ./data\archiveII\processed


In [3]:
dummy_csv_path = os.path.join(PROCESSED_DIR, "archiveII_dummy.csv")

# Tiny toy dataset mimicking ArchiveII:
toy_data = [
    {
        "sequence_id": "toy_1",
        "sequence": "GCAUCU",
        "dot_bracket": "..(..)"   # simple valid structure
    },
    {
        "sequence_id": "toy_2",
        "sequence": "AUGCGAU",
        "dot_bracket": "(.().)."  # another valid structure
    }
]

toy_df = pd.DataFrame(toy_data)
toy_df.to_csv(dummy_csv_path, index=False)

print("Dummy ArchiveII-like CSV saved to:", dummy_csv_path)
toy_df


Dummy ArchiveII-like CSV saved to: ./data\archiveII\processed\archiveII_dummy.csv


,sequence_id,sequence,dot_bracket
0,toy_1,GCAUCU,..(..)
1,toy_2,AUGCGAU,(.().).


# Step 3 – Encoding Utilities

In this notebook we implement:

1. `encode_sequence(seq)` → integer IDs for A/C/G/U  
2. `dotbracket_to_pairs(dot_bracket)` → list of base pairs `(i, j)`  
3. `pairs_to_matrix(pairs, length)` → L × L matrix  
4. `dotbracket_to_matrix(dot_bracket)` → convenience wrapper


In [6]:
from typing import List, Tuple
import numpy as np
import torch


In [11]:
# Step 3: Encoding Utilities

BASE2IDX = {"A": 0, "C": 1, "G": 2, "U": 3}


def encode_sequence(seq: str) -> torch.Tensor:
    """
    Map A/C/G/U -> 0/1/2/3.
    Unknown bases can be mapped to 0 (A) for now.
    """
    idxs = [BASE2IDX.get(b, 0) for b in seq]
    return torch.tensor(idxs, dtype=torch.long)


def dotbracket_to_pairs(dot_bracket: str) -> List[Tuple[int, int]]:
    """
    Convert dot-bracket notation to a list of base pairs (i, j), 0-based.

    '(' means open, ')' means close, '.' means unpaired.
    We assume a simple non-pseudoknotted structure.
    """
    stack = []
    pairs = []
    for i, ch in enumerate(dot_bracket):
        if ch == "(":
            stack.append(i)
        elif ch == ")":
            if not stack:
                raise ValueError(f"Unmatched closing parenthesis at position {i}")
            j = stack.pop()
            # Pair (j, i)
            pairs.append((j, i))
        else:
            # '.' or other unpaired symbol
            continue

    if stack:
        raise ValueError("Unmatched opening parenthesis in dot-bracket string")

    return pairs


def pairs_to_matrix(pairs: List[Tuple[int, int]], length: int) -> np.ndarray:
    """
    Turn a list of base pairs into a symmetric L x L matrix.
    """
    mat = np.zeros((length, length), dtype=np.float32)
    for i, j in pairs:
        if 0 <= i < length and 0 <= j < length:
            mat[i, j] = 1.0
            mat[j, i] = 1.0
    return mat


def dotbracket_to_matrix(dot_bracket: str) -> np.ndarray:
    """
    Convenience wrapper: dot-bracket -> pair list -> L x L matrix.
    """
    length = len(dot_bracket)
    pairs = dotbracket_to_pairs(dot_bracket)
    return pairs_to_matrix(pairs, length)


In [8]:
example_seq = "GCAUCU"
example_db = "..(..)"

print("Sequence:", example_seq)
print("Dot-bracket:", example_db)

encoded_seq = encode_sequence(example_seq)
pair_list = dotbracket_to_pairs(example_db)
pair_mat = dotbracket_to_matrix(example_db)

print("Encoded sequence:", encoded_seq)
print("Pair list:", pair_list)
print("Pair matrix:\n", pair_mat)


Sequence: GCAUCU
Dot-bracket: ..(..)
Encoded sequence: tensor([2, 1, 0, 3, 1, 3])
Pair list: [(2, 5)]
Pair matrix:
 [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]


# Step 4 – ArchiveII-Style Dataset & Collate Function

This notebook:

- Defines an `ArchiveIIDataset` that reads a CSV with:
  - `sequence_id`
  - `sequence`
  - `dot_bracket`
- Uses the encoding utilities to turn dot-bracket into a base-pair matrix
- Defines a `collate_rna_batch` function for variable-length sequences
- Tests a small DataLoader using the **dummy CSV** from Step 2


## Step 4: ArchiveII-style Dataset + Collate Function

We will:

1. Define an `ArchiveIIDataset` class that:
   - loads a CSV containing `sequence_id, sequence, dot_bracket`
   - returns `(sequence_str, base_pair_matrix)` for each example  

2. Define a `collate_rna_batch` function that:
   - converts a list of `(seq_str, mat)` into:
     - `input_ids` : (B, Lmax)
     - `mask`      : (B, Lmax)  (True for real tokens, False for padding)
     - `targets`   : (B, Lmax, Lmax)

Here, we will only use the **dummy CSV** created earlier, but the code should work with real ArchiveII data once you generate the CSV.


In [12]:
class ArchiveIIDataset(Dataset):
    """
    Minimal ArchiveII-style dataset.

    Expects a CSV with columns:
      - sequence_id
      - sequence
      - dot_bracket
    """

    def __init__(self, csv_path: str):
        self.df = pd.read_csv(csv_path)
        if not {"sequence", "dot_bracket"}.issubset(self.df.columns):
            raise ValueError("CSV must contain 'sequence' and 'dot_bracket' columns")

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        seq = row["sequence"]
        db = row["dot_bracket"]
        # Convert dot-bracket to base-pair matrix
        mat = dotbracket_to_matrix(db)  # numpy array (L, L)
        return seq, torch.from_numpy(mat)  # return as torch tensor


# Instantiate dataset using the dummy CSV
archive_dataset = ArchiveIIDataset(dummy_csv_path)
len(archive_dataset), archive_dataset[0]


(2,
 ('GCAUCU', tensor([[0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 1.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 1., 0., 0., 0.]])))

In [13]:
def collate_rna_batch(batch):
    """
    Collate function for variable-length RNA sequences.

    Args:
        batch: list of (seq_str, base_pair_matrix_tensor (L, L))

    Returns:
        input_ids: (B, Lmax) int64
        mask:      (B, Lmax) bool
        targets:   (B, Lmax, Lmax) float32
    """
    seqs, mats = zip(*batch)
    lengths = [len(s) for s in seqs]
    Lmax = max(lengths)
    B = len(seqs)

    input_ids = torch.zeros((B, Lmax), dtype=torch.long)
    mask = torch.zeros((B, Lmax), dtype=torch.bool)
    targets = torch.zeros((B, Lmax, Lmax), dtype=torch.float32)

    for i, (seq, mat) in enumerate(zip(seqs, mats)):
        L = len(seq)
        input_ids[i, :L] = encode_sequence(seq)
        mask[i, :L] = True
        targets[i, :L, :L] = mat

    return input_ids, mask, targets


# Test DataLoader
loader = DataLoader(archive_dataset, batch_size=2, shuffle=False, collate_fn=collate_rna_batch)

for batch in loader:
    input_ids, mask, targets = batch
    print("input_ids shape:", input_ids.shape)
    print("mask shape:", mask.shape)
    print("targets shape:", targets.shape)
    print("input_ids:", input_ids)
    print("mask:", mask)
    break


input_ids shape: torch.Size([2, 7])
mask shape: torch.Size([2, 7])
targets shape: torch.Size([2, 7, 7])
input_ids: tensor([[2, 1, 0, 3, 1, 3, 0],
        [0, 3, 2, 1, 2, 0, 3]])
mask: tensor([[ True,  True,  True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True,  True,  True]])


## Step 5: Nussinov Dynamic Programming Decoder (Baseline)

We implement the classical Nussinov algorithm:

- For a given RNA sequence, find a structure that:
  - obeys base-pairing rules (A–U, G–C, G–U)
  - has no crossing pairs (no pseudoknots)
  - maximizes the number of base pairs (or an equivalent score)

This gives us a **DP baseline** and a **decoder** that we will reuse:

- `nussinov_from_sequence(seq)` → dot-bracket (classical, equal score per valid pair)
- `nussinov_from_scores(seq, score_matrix)` → dot-bracket, where `score_matrix[i,j]`
  can come from a neural network’s base-pair probabilities/logits.


In [ ]:
# Base-pair compatibility (Watson-Crick + wobble)
def can_pair(b1: str, b2: str) -> bool:
    """
    Return True if two bases can form a canonical pair (A-U, G-C, G-U).
    """
    pair = {b1 + b2, b2 + b1}
    return bool(pair & {"AU", "UA", "GC", "CG", "GU", "UG"})


In [15]:
def nussinov_dp(seq: str) -> Tuple[np.ndarray, np.ndarray]:
    """
    Classical Nussinov dynamic programming (max number of base pairs).

    Returns:
        dp: 2D array of scores (L x L)
        trace: 2D array of traceback decisions (optional / for debugging)
    """
    L = len(seq)
    dp = np.zeros((L, L), dtype=np.float32)

    # Optional: store decisions if you want more detailed traceback info
    # For simplicity, we will not store a full trace matrix here.

    # Fill DP table by increasing subsequence length
    for length in range(1, L):  # length = j - i
        for i in range(L - length):
            j = i + length
            # Case 1: i unpaired
            best = dp[i + 1, j]
            # Case 2: j unpaired
            best = max(best, dp[i, j - 1])
            # Case 3: i paired with j
            if can_pair(seq[i], seq[j]):
                best = max(best, dp[i + 1, j - 1] + 1)
            # Case 4: bifurcation
            for k in range(i + 1, j):
                best = max(best, dp[i, k] + dp[k + 1, j])

            dp[i, j] = best

    return dp, None  # we won't use 'trace' here


def nussinov_traceback(seq: str, dp: np.ndarray, i: int, j: int, pairs: List[Tuple[int, int]]):
    """
    Recursive traceback to recover base pairs from the DP table.

    Modifies 'pairs' in place.
    """
    if i >= j:
        return

    # Same logic as fill, but we compare dp entries
    if dp[i, j] == dp[i + 1, j]:
        nussinov_traceback(seq, dp, i + 1, j, pairs)
    elif dp[i, j] == dp[i, j - 1]:
        nussinov_traceback(seq, dp, i, j - 1, pairs)
    elif can_pair(seq[i], seq[j]) and dp[i, j] == dp[i + 1, j - 1] + 1:
        pairs.append((i, j))
        nussinov_traceback(seq, dp, i + 1, j - 1, pairs)
    else:
        # bifurcation
        for k in range(i + 1, j):
            if dp[i, j] == dp[i, k] + dp[k + 1, j]:
                nussinov_traceback(seq, dp, i, k, pairs)
                nussinov_traceback(seq, dp, k + 1, j, pairs)
                break


def pairs_to_dotbracket(pairs: List[Tuple[int, int]], length: int) -> str:
    """
    Convert a list of base pairs into dot-bracket notation.
    """
    chars = ["."] * length
    for i, j in pairs:
        chars[i] = "("
        chars[j] = ")"
    return "".join(chars)


def nussinov_from_sequence(seq: str) -> str:
    """
    Run classical Nussinov on a sequence and return dot-bracket structure.
    """
    L = len(seq)
    if L == 0:
        return ""
    dp, _ = nussinov_dp(seq)
    pairs: List[Tuple[int, int]] = []
    nussinov_traceback(seq, dp, 0, L - 1, pairs)
    return pairs_to_dotbracket(pairs, L)


In [16]:
# Test Nussinov on a simple toy sequence

toy_seq = "GCAUCU"
pred_db = nussinov_from_sequence(toy_seq)

print("Sequence:      ", toy_seq)
print("Pred structure:", pred_db)

# Convert predicted structure to matrix and print
pred_pairs = dotbracket_to_pairs(pred_db) if "(" in pred_db or ")" in pred_db else []
pred_mat = pairs_to_matrix(pred_pairs, len(toy_seq))

print("Pred pairs:", pred_pairs)
print("Pred matrix:\n", pred_mat)


Sequence:       GCAUCU
Pred structure: ()()..
Pred pairs: [(0, 1), (2, 3)]
Pred matrix:
 [[0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


## ✅ Up to Step 5 Completed

In this notebook, we have:

1. **Project config & imports** (Step 1)
2. **Data directory & dummy ArchiveII-style CSV** (Step 2)
3. **Encoding utilities**:
   - `encode_sequence`
   - `dotbracket_to_pairs`
   - `dotbracket_to_matrix`  (Step 3)
4. **ArchiveIIDataset + collate function** for PyTorch (Step 4)
5. **Nussinov DP decoder** (`nussinov_from_sequence`) as a classical baseline (Step 5)

---

### Next steps (beyond this notebook)

- Implement `nussinov_from_scores(seq, score_matrix)` to decode neural outputs.
- Add RNN / BiLSTM / Transformer models and plug them into this data pipeline.
- Implement evaluation metrics (PPV, Sensitivity, F1) using predicted vs true base pairs.
- Replace the dummy CSV with real ArchiveII / bpRNA-NF-15.0 processed files.
